## 2. High-level Pseudocode

**WARNING:** This uses one-based indexing. Keep that constantly in mind when implementing and change all index-based logic accordingly.




    list<pair<int, int>> convexHull(list<pair<int, int>> points) {
        
        // Find a centroid point amongst the given points
        centroid = getCentroid(points)

        // Define a reference point relative to the centroid
        ref = getReference(centroid)

        // For every point, process its interior angle. Additionally, process and find the lowest point.
        pairs = []
        lowestPoint = None
        lowestPair = None
        for point in points {
            if point != centroid {
                // First, define which region the point falls
                left = point.x <= centroid.x

                // 
                intAngle = getInteriorAngle(centroid, point, ref)
                if (!left) {
                    intAngle = 360 - intAngle
                }
                Pair<Point, Angle> pt_ang_pair = Pair(point, intAngle)
                pairs.append(pt_ang_pair)

                // Determine if the point is the lowest point
                if lowestPoint == None || point.y < lowestPoint.y || (point.y == lowestPoint.y && point.x < lowestPoint.x) {
                    lowestPoint = point
                    lowestPair = pt_ang_pair
                }
            }
        }

        // Sort the pairs
        sort(pairs)

        // Find the lowest point
        lowestPointIndex = 1
        while pairs[lowestPointIndex] != lowestPair {
            lowestPointIndex += 1
        }

        // Rotate the list to start at the lowest point
        pair_order = []
        for newIndex in range(1, len(pairs) + 1) {
            oldIndex = (newIndex + lowestPointIndex - 1) % len(pairs) + 1
            pair_order.append(pairs[oldIndex])
        }

        // For each pair, form a quadrilateral, and determine if the interior angle is convex

        // The currently-evaluated point will be part of return_points because it might not be
        //     the point immediately before the next point in pair_order
        // That said, i will still point one before the next point to make the loop logic easier
        //     (so we end at the last index)
        
        return_points = [pair_order[1], pair_order[2]]
        i = 2
        while i <= len(pair_order) {
            currPair = return_points.pop()
            prevPair = return_points[-1]
            
            if i == len(pair_order) {
                nextPair = pair_order[1]
            } else {
                nextPair = pair_order[i + 1]
            }
           
            intAngle = getInteriorAngleQuad(prevPair, currPair, nextPair, centroid)

            // Convex?
            if intAngle < 180 {
                // Add back the current point
                return_points.append(currPair)
                
                // Do the next point next
                return_points.append(nextPair)
                
                // Increment i to move forward
                i += 1
            } else if i == 2 {
                // Don't add back the current point (it's not in the hull)
                
                // Still do the next point next (since we can't step back)
                return_points.append(nextPair)
                
                // Increment i to move forward
                i += 1
            } else {
                // Don't add back the current point (it's not in the hull)
                
                // We're stepping back, so don't do the next point yet
                
                // Don't increment i, the following point hasn't changed
            }
        }

        // The first element of return_points was pushed in again, pop it off
        return_points.pop()
        
        // Extract the points from return_points
        convex_hull = []
        for pair in return_points:
            convex_hull.append(pair.point)

        return convex_hull
    }



In [5]:
import math

In [6]:
# Point class
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

# Point-Angle-Pair class
class Pair:
    def __init__(self, point: Point, angle):
        self.point = point
        self.angle = angle

# Define some helper functions
def getCentroid(points: list) -> Point:
    avgX = 0
    avgY = 0
    for point in points:
        avgX += point.x
        avgY += point.y
    avgX /= len(points)
    avgY /= len(points)

    return Point(avgX, avgY)

def getReference(centroid: Point) -> Point:
    return Point(centroid.x, centroid.y - 10)

def getInteriorAngle(a: Point, b: Point, c: Point):
    return math.degrees(math.atan2(c.y-b.y, c.x-b.x) - math.atan2(a.y-b.y, a.x-b.x))

def getInteriorAngleQuad(a: Point, b: Point, c: Point, d: Point):
    angleLeft = getInteriorAngle(a, b, d)
    angleRight = getInteriorAngle(d, b, c)
    return angleLeft + angleRight

def sortPointAnglePairs(pairs: list):
    pairs.sort(key=lambda p: p.angle)

def convexHullEdgePoints(points: list):
    # Edge case catching
    if len(points) == 3:
        return points

    # Define a centroid
    centroid = getCentroid(points)

    # Define a reference point
    ref = getReference(centroid)

    pairs = []
    lowestPoint = None
    lowestPair = None
    for point in points:
        if point.x != centroid.x and point.y != centroid.y:
            # First, determine the left/right region of the centroid
            isLeft = point.x < centroid.x

            # Then find the interior angle
            intAngle = getInteriorAngle(centroid, point, ref)

            # Adjust for placement about the centroid
            if (not isLeft):
                intAngle = 360 - intAngle
            pt_ang_pair = Pair(point, intAngle)
            pairs.append(pt_ang_pair)

            # Determine if the point is the lowest point
            if lowestPoint == None or point.y < lowestPoint.y or (point.y == lowestPoint.y and point.x < lowestPoint.x):
                lowestPoint = point
                lowestPair = pt_ang_pair
            
    # Sort the pairs
    sortPointAnglePairs(pairs)

    # Find the lowest point
    lowestPointIndex = 0
    while pairs[lowestPointIndex] != lowestPair:
        lowestPointIndex += 1

    # Rotate the list to start at the lowest point
    pair_order = []
    for newIndex in range(0, len(pairs)):
        oldIndex = (newIndex + lowestPointIndex) % len(pairs)
        pair_order.append(pairs[oldIndex])

    # For each pair, form a quadrilateral, and determine if the interior angle is convex
    # The currently-evaluated point will be part of return_points because it might not be
    #     the point immediately before the next point in pair_order
    # That said, i will still point one before the next point to make the loop logic easier
    #     (so we end at the last index)
    return_points = [pair_order[0], pair_order[1]]
    i = 1
    while i < len(pair_order):
        currPair = return_points.pop()
        prevPair = return_points[-1]
        
        
        if i == len(pair_order) - 1:
            nextPair = pair_order[0]
        else:
            nextPair = pair_order[i + 1]
        
        intAngle = getInteriorAngleQuad(prevPair.point, currPair.point, nextPair.point, centroid)

        # Check: Convex?
        if intAngle < 180:
            # Add back the current point
            return_points.append(currPair)
            
            # Do the next point next
            return_points.append(nextPair)
            
            # Increment i to move forward
            i += 1
        elif i == 1:
            # Don't add back the current point (it's not in the hull)
            return_points.append(nextPair)
            
            # Increment i to move forward
            i += 1

    # The first element of return_points was pushed in again, pop it off
    return_points.pop()
    
    # Extract the points from return_points
    convex_hull = []
    for pair in return_points:
        convex_hull.append(pair.point)

    return convex_hull
